<a href="https://colab.research.google.com/github/chanmeee/settings/blob/main/PyTorch_reproducibility.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Reproducible PyTorch를 위한 randomness 올바르게 제어하기
## PyTorch의 완벽한 실험 재현을 위해 고려해야 할 randomness

- 참고
    - PyTorch의 [공식 Reproducibility 문서](https://pytorch.org/docs/stable/notes/randomness.html)

---

### **PyTorch 실험의 재현 가능성을 위해 고려해야 할 4가지 요소**

### 1. PyTorch Randomness
- 메인 프레임워크인 PyTorch의 random seed를 고정
- 다만 CUDA 함수를 사용하는 **PyTorch 함수들 중에 nondeterministic한 함수**가 존재하며, 이러한 함수들을 사용할 경우 완벽히 재현하는 것이 현재로썬 불가능
    - (예시) atomicAdd 연산이 포함되어있는 함수들
        - **(forward kernel)** torch.Tensor.index_add_() , torch.Tensor.scatter_add_() , torch.bincount()
        - **(backward kernel)** torch.nn.functional.embedding_bag() , torch.nn.functional.ctc_loss() , torch.nn.functional.interpolate()

### 2. CuDNN
- CuDNN: 딥러닝에 특화된 CUDA library
- 단, **연산 처리 속도가 감소**되는 부작용 발생
- 따라서 모델 구현/실험하는 초기 단계보다는 모델 배포하는 연구 후반 단계에 사용하는 것을 권장

### 3. Numpy
- Scikit-learn 등 딥러닝에 도움이 되는 주요 라이브러리도 Numpy를 backend로 사용하고 있고, PyTorch로 코드를 짤 때 대부분 Numpy로 data를 받아오고, metric을 계산하는 경우가 많아서 Numpy의 random seed도 고정을 해주어야 함

### 4. Random
- torchvision의 transforms 함수는 torch, numpy, cudnn이 아닌 python random 라이브러리에 의해 randomness가 결정 
    - RandomCrop(), RandomHorizontalFlip() 함수 등에서도 randomness가 존재
- 따라서 random 라이브러리도 randomness를 제어해야 함

---
### 최종 코드

In [ ]:
import torch 
import numpy as np
import random

torch.manual_seed(random_seed)
torch.cuda.manual_seed(random_seed)
torch.cuda.manual_seed_all(random_seed) # if use multi-GPU
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
np.random.seed(random_seed)
random.seed(random_seed)